In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
import pandas as pd

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
db = client.vg_reviews_db

In [4]:
db.drop_collection("rev_scrape")

{'ok': 0.0,
 'errmsg': 'ns not found',
 'code': 26,
 'codeName': 'NamespaceNotFound'}

In [ ]:
collection = db.rev_scrape

In [12]:
sdf = pd.read_csv("vgsales.csv")
sdf.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [ ]:
sdf["Platform"].value_counts()

Not all of these platforms are available on metacritic.com
This is a list of the platforms that will be dropped from this data in order to scrape from metacritic
NES, SNES, SAT, 2600, GB, GEN, NG, SCD, WS, 3DO, TG16, GG, PCFX

In [6]:
def urlmaker(platform, name):
    platform_dict = {
        "ds": "ds",
        "ps2":"playstation-2",
        "ps3": "playstation-3",
        "wii": "wii",
        "x360": "xbox-360",
        "psp": "psp",
        "ps": "playstation",
        "pc": "pc",
        "xb": "xbox",
        "gba": "game-boy-advance",
        "gc": "gamecube",
        "3ds": "3ds",
        "psv": "playstation-vita",
        "ps4": "playstation-4",
        "n64": "nintendo-64",
        "xone": "xbox-one",
        "wiiu": "wii-u",
        "dc": "dreamcast",
    }
    
    platform_fixed = platform_dict[platform.lower()]
    
    name_fixed = name.replace(" ", "-")
    name_fixed = name_fixed.lower()
    
    return f"http://metacritic.com/game/{platform_fixed}/{name_fixed}"

In [ ]:
# url_list = []
# for row in sdf[["Platform", "Name"]].iterrows():
#     try:
#         url = urlmaker(row[1]["Platform"], row[1]["Name"])
#         url_list.append(url)
#     except: 
#         pass
    
# url_list

In [7]:
gamelist = []
for row in sdf[["Platform", "Name"]].iterrows():
    try:
        url = urlmaker(row[1]["Platform"], row[1]["Name"])
        gamedict = {
            "url": url,
            "Title": row[1]["Name"],
            "Platform": row[1]["Platform"]
        }
        gamelist.append(gamedict)
    except: 
        pass

In [ ]:
response = requests.request("GET", url_list[0], headers={"User-Agent": "Mozilla/5.0"})
soup = BeautifulSoup(response.text, "lxml")
reviews = soup.find_all("li", class_="critic_review")
# reviews[0].find("a", class_="external").text

In [ ]:
reviews[0].find("div", class_="review_body").text.strip()

In [ ]:
# rdf_list = []
# for game_url in url_list:
#     try:
#         response = requests.request("GET", game_url, headers={"User-Agent": "Mozilla/5.0"})
#         soup = BeautifulSoup(response.text, "lxml")
        
#         reviews = soup.find_all("li", class_="critic_review")
        
#         for review in reviews:
            
#             title = soup.find("div", class_="product_title").find("h1").text
#             source = review.find("div", class_="source").text
#             score = review.find("div", class_="review_grade").text.strip()
#             body =  review.find("div", class_="review_body").text.strip()
            
#             if (title and source and score and body):
#                 print("---------------------")
#                 print(title)
#                 print(source)
#                 print(score)
#                 print(body)
                
#             review_data = {
#                 "Game Title": title,
#                 "Review Source": source,
#                 "Review Score": score,
#                 "Review Body": body
#             }
            
#             rdf_list.append(review_data)
#     except Exception as e:
#         print(e)
            

In [10]:
rdf_list = []
for game in gamelist:
    try:
        response = requests.request("GET", game["url"], headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(response.text, "lxml")
        
        reviews = soup.find_all("li", class_="critic_review")
        
        for review in reviews:
            
#             title = soup.find("div", class_="product_title").find("h1").text
            source = review.find("div", class_="source").text
            score = review.find("div", class_="review_grade").text.strip()
            body =  review.find("div", class_="review_body").text.strip()
            
            if (source and score and body):
                print("---------------------")
                print(game["Title"])
                print(game["Platform"])
                print(source)
                print(score)
                print(body)
                
            review_data = {
                "Game Title": game["Title"],
                "Platform": game["Platform"],
                "Review Source": source,
                "Review Score": score,
                "Review Body": body
            }
            
            rdf_list.append(review_data)
    except Exception as e:
        print(e)

---------------------
Wii Sports
Wii
Official Nintendo Magazine UK
90
The ideal way to get into Wii gaming. It's funny, simple and approachable, and gamers and non-gamers alike should get something out of it. [Christmas 2006, p.62]
---------------------
Wii Sports
Wii
GameZone
85
Sure, it lacks goals and can be beaten very quickly. But the ultra-responsive technology and high multiplayer replay value are far greater than any other party or sports game collection.
---------------------
Wii Sports
Wii
GameDaily
80
Wii Sports certainly won't be hailed as the most realistic sports sim on the market, but it's the best thing - next to a bottle of tequila - to take to a friend's holiday house party.
---------------------
Wii Sports
Wii
GamingExcellence
77
Despite these two solo modes, Wii Sports is really all about getting some friends together and playing a few matches of tennis or bowling. The game isn't designed to be a true authentic sports simulation, it's all about the sheer entertainme

---------------------
Wii Fit
Wii
Pelit (Finland)
92
An excellent combination of honest training and highly entertaining partygames. Losing weight has never been this much fun. [May 2008]
---------------------
Wii Fit
Wii
GameFocus
89
But none of these activities would be possible if it wasn’t because of the Wii Balance Board, which, in my opinion, is one of the best interactive peripherals ever invented.
---------------------
Wii Fit
Wii
NZGamer
85
If you’re able to motivate yourself to exercise you can do no wrong in getting Wii Fit. Granted it’s not for everyone, but the ones it is for will be able to take a lot away from it.
---------------------
Wii Fit
Wii
GameSpy
80
Wii Fit won't replace your favorite videogames, but it may become an activity you'll want to do for a few minutes in the morning before school or work.
---------------------
Wii Fit
Wii
PTGamers
80
Wii Fit has its highs and lows, but comprises a range of games that encourages us to return to the workout. It's not for

KeyboardInterrupt: 

In [ ]:
collection.insert_many(rdf_list)

In [11]:
rdf = pd.DataFrame(rdf_list)
rdf.head()

,Game Title,Platform,Review Source,Review Score,Review Body
0,Wii Sports,Wii,Official Nintendo Magazine UK,90,The ideal way to get into Wii gaming. It's fun...
1,Wii Sports,Wii,GameZone,85,"Sure, it lacks goals and can be beaten very qu..."
2,Wii Sports,Wii,GameDaily,80,Wii Sports certainly won't be hailed as the mo...
3,Wii Sports,Wii,GamingExcellence,77,"Despite these two solo modes, Wii Sports is re..."
4,Wii Sports,Wii,My Gamer,74,This is one you can feel confident about showi...


In [ ]:
rdf.info()